
<br>
==================<br>
Two-class AdaBoost<br>
==================<br>
This example fits an AdaBoosted decision stump on a non-linearly separable<br>
classification dataset composed of two "Gaussian quantiles" clusters<br>
(see :func:`sklearn.datasets.make_gaussian_quantiles`) and plots the decision<br>
boundary and decision scores. The distributions of decision scores are shown<br>
separately for samples of class A and B. The predicted class label for each<br>
sample is determined by the sign of the decision score. Samples with decision<br>
scores greater than zero are classified as B, and are otherwise classified<br>
as A. The magnitude of a decision score determines the degree of likeness with<br>
the predicted class label. Additionally, a new dataset could be constructed<br>
containing a desired purity of class B, for example, by only selecting samples<br>
with a decision score above some value.<br>


In [ ]:
print(__doc__)

Author: Noel Dawe <noel.dawe@gmail.com><br>
<br>
License: BSD 3 clause

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_gaussian_quantiles

Construct dataset

In [ ]:
X1, y1 = make_gaussian_quantiles(cov=2.,
                                 n_samples=200, n_features=2,
                                 n_classes=2, random_state=1)
X2, y2 = make_gaussian_quantiles(mean=(3, 3), cov=1.5,
                                 n_samples=300, n_features=2,
                                 n_classes=2, random_state=1)
X = np.concatenate((X1, X2))
y = np.concatenate((y1, - y2 + 1))

Create and fit an AdaBoosted decision tree

In [ ]:
bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                         algorithm="SAMME",
                         n_estimators=200)

In [ ]:
bdt.fit(X, y)

In [ ]:
plot_colors = "br"
plot_step = 0.02
class_names = "AB"

In [ ]:
plt.figure(figsize=(10, 5))

Plot the decision boundaries

In [ ]:
plt.subplot(121)
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                     np.arange(y_min, y_max, plot_step))

In [ ]:
Z = bdt.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
cs = plt.contourf(xx, yy, Z, cmap=plt.cm.Paired)
plt.axis("tight")

Plot the training points

In [ ]:
for i, n, c in zip(range(2), class_names, plot_colors):
    idx = np.where(y == i)
    plt.scatter(X[idx, 0], X[idx, 1],
                c=c, cmap=plt.cm.Paired,
                s=20, edgecolor='k',
                label="Class %s" % n)
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.legend(loc='upper right')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Decision Boundary')

Plot the two-class decision scores

In [ ]:
twoclass_output = bdt.decision_function(X)
plot_range = (twoclass_output.min(), twoclass_output.max())
plt.subplot(122)
for i, n, c in zip(range(2), class_names, plot_colors):
    plt.hist(twoclass_output[y == i],
             bins=10,
             range=plot_range,
             facecolor=c,
             label='Class %s' % n,
             alpha=.5,
             edgecolor='k')
x1, x2, y1, y2 = plt.axis()
plt.axis((x1, x2, y1, y2 * 1.2))
plt.legend(loc='upper right')
plt.ylabel('Samples')
plt.xlabel('Score')
plt.title('Decision Scores')

In [ ]:
plt.tight_layout()
plt.subplots_adjust(wspace=0.35)
plt.show()